**MOUNT GOOGLE DRIVE**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**DATASET DOWNLOAD**

In [ ]:
!pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="TCMa2v8HPfvxgPmxuUYK")
project = rf.workspace("my-workspace-fq11i").project("car-and-person-identification")
version = project.version(5)

# Specify the custom download path
dataset = version.download("yolov8", location="/content/drive/MyDrive/Intern/Insiso/dataset")


In [ ]:
import os

base_path = "/content/drive/MyDrive/Intern/Insiso/dataset"

train_path = os.path.join(base_path, "train/images")
val_path   = os.path.join(base_path, "valid/images")
test_path  = os.path.join(base_path, "test/images")

def count_images(folder):
    image_extensions = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
    return len([f for f in os.listdir(folder) if f.lower().endswith(image_extensions)])

train_count = count_images(train_path)
val_count   = count_images(val_path)
test_count  = count_images(test_path)

print(f"Train images: {train_count}")
print(f"Validation images: {val_count}")
print(f"Test images: {test_count}")


Train images: 354
Validation images: 99
Test images: 62


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.2 MB/s eta 0:00:00


**TRAIN**

In [14]:
from ultralytics import YOLO
import shutil
import os

#Load YOLOv8 base model
model = YOLO('yolov8n.pt')

# runs
runs_drive_path = "/content/drive/MyDrive/Intern/Insiso/runs"

# Training the model
model.train(
    data="/content/drive/MyDrive/Intern/Insiso/dataset/data.yaml",
    epochs=30,
    imgsz=640,
    batch=8,
    workers=2,
    optimizer='Adam',
    project=runs_drive_path,
    name='yolov8n_adas',
    lr0=0.001,
    lrf=0.01,
    augment=True
)

# Save trained model in Drive
model.save('/content/drive/MyDrive/Intern/Insiso/models/model_30epochs.pt')

# Evaluate model
metrics = model.val(project=runs_drive_path, name='yolov8n_adas_val')
print(f"Validation metrics after training: {metrics}")

# save runs in drive
detect_src = "/content/runs/detect"
detect_dest = os.path.join(runs_drive_path, "detect")
if os.path.exists(detect_src):
    shutil.copytree(detect_src, detect_dest, dirs_exist_ok=True)
    print(f"✅ Detection results copied to: {detect_dest}")
else:
    print("⚠️ No detect folder found yet. Run inference before copying.")


Ultralytics 8.3.206 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/Intern/Insiso/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8n_adas, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=100, per

**TEST**

In [16]:
import cv2
from ultralytics import YOLO
import os
import yaml
import torch
import numpy as np


IOU_THRESHOLD = 0.5
CONFIDENCE_THRESHOLD = 0.25

#  IoU Calculation
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea + 1e-6)
    return iou

# Box Drawing for Debugging
def draw_boxes(image, boxes, class_names):
    for box in boxes:
        if len(box) < 6:
            continue
        x1, y1, x2, y2, label, conf = box
        if not (0 <= int(label) < len(class_names)):
            continue
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"{class_names[int(label)]} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    return image

# Matching Predictions with Ground Truth
def match_predictions(gt_boxes, pred_boxes):
    matched_gt = set()
    tp = 0
    for pred in pred_boxes:
        best_iou = 0
        best_gt_idx = -1
        for idx, gt in enumerate(gt_boxes):
            if idx in matched_gt:
                continue
            iou = calculate_iou(pred[:4], gt[:4])
            if iou > best_iou and iou >= IOU_THRESHOLD and pred[4] == gt[4]:
                best_iou = iou
                best_gt_idx = idx
        if best_gt_idx >= 0:
            matched_gt.add(best_gt_idx)
            tp += 1
    fp = len(pred_boxes) - tp
    fn = len(gt_boxes) - tp
    return tp, fp, fn

#  Core Evaluation Logic
def detect_and_evaluate(image_folder, label_folder, class_names, output_folder):
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.jpg')]

    total_tp = 0
    total_fp = 0
    total_fn = 0

    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)
        if image is None:
            continue

        height, width = image.shape[:2]
        results = model(image)

        label_file = os.path.join(label_folder, image_file.replace('.jpg', '.txt'))
        if not os.path.exists(label_file):
            continue

        # Ground Truth Boxes
        gt_boxes = []
        with open(label_file, 'r') as lf:
            for line in lf:
                parts = line.strip().split()
                if len(parts) == 5:
                    class_id, x_center, y_center, w, h = map(float, parts)
                    x_min = int((x_center - w / 2) * width)
                    y_min = int((y_center - h / 2) * height)
                    x_max = int((x_center + w / 2) * width)
                    y_max = int((y_center + h / 2) * height)
                    gt_boxes.append([x_min, y_min, x_max, y_max, int(class_id)])

        # Predicted Boxes
        pred_boxes = []
        for box in results[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            conf = float(box.conf[0])
            if conf < CONFIDENCE_THRESHOLD:
                continue
            label = int(box.cls[0].item()) if isinstance(box.cls[0], torch.Tensor) else int(box.cls[0])
            pred_boxes.append([x1, y1, x2, y2, label, conf])

        # Match and accumulate
        tp, fp, fn = match_predictions(gt_boxes, pred_boxes)
        total_tp += tp
        total_fp += fp
        total_fn += fn

        # Save visualization
        output_image = draw_boxes(image.copy(), pred_boxes, class_names)
        cv2.imwrite(os.path.join(output_folder, image_file), output_image)

    # Metrics Computation
    precision = total_tp / (total_tp + total_fp + 1e-6)
    recall = total_tp / (total_tp + total_fn + 1e-6)
    f1 = 2 * precision * recall / (precision + recall + 1e-6)
    accuracy = total_tp / (total_tp + total_fp + total_fn + 1e-6)

    print(f"\n--- Evaluation Metrics ---")
    print(f"True Positives: {total_tp}")
    print(f"False Positives: {total_fp}")
    print(f"False Negatives: {total_fn}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Testing Accuracy (IoU-based): {accuracy:.4f}")

# Load Model and Dataset Configuration

custom_model_path = '/content/drive/MyDrive/Intern/Insiso/models/model_30epochs.pt'
model = YOLO(custom_model_path)

data_yaml_path = '/content/drive/MyDrive/Intern/Insiso/dataset/data.yaml'
with open(data_yaml_path, 'r') as file:
    data_config = yaml.safe_load(file)
class_names = data_config['names']

test_image_folder = '/content/drive/MyDrive/Intern/Insiso/dataset/test/images'
test_label_folder = '/content/drive/MyDrive/Intern/Insiso/dataset/test/labels'
output_folder = '/content/drive/MyDrive/Intern/Insiso/outputs/output30epochs'
os.makedirs(output_folder, exist_ok=True)

#  Run Evaluation
detect_and_evaluate(test_image_folder, test_label_folder, class_names, output_folder)



0: 640x640 1 car, 1 person, 9.1ms
Speed: 2.1ms preprocess, 9.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 2 persons, 11.5ms
Speed: 2.7ms preprocess, 11.5ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 9.6ms
Speed: 2.5ms preprocess, 9.6ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 persons, 9.2ms
Speed: 2.2ms preprocess, 9.2ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 3 cars, 4 persons, 9.0ms
Speed: 2.4ms preprocess, 9.0ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 6 cars, 2 persons, 14.5ms
Speed: 2.7ms preprocess, 14.5ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 car, 1 person, 33.8ms
Speed: 2.6ms preprocess, 33.8ms inference, 7.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 7 persons, 8.7ms
Speed: 2.3ms preprocess, 8.7ms inference, 1.8ms postprocess

**CONVERT TO ONNX**

In [17]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("/content/drive/MyDrive/Intern/Insiso/models/model_30epochs.pt")

# Export to ONNX
onnx_path = "/content/drive/MyDrive/Intern/Insiso/models/model.onnx"
model.export(format="onnx", opset=12, dynamic=True)

print(f"✅ Model exported to ONNX: {onnx_path}")


Ultralytics 8.3.206 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.00GHz)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/drive/MyDrive/Intern/Insiso/models/model_30epochs.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)

ONNX: starting export with onnx 1.19.1rc1 opset 12...
ONNX: slimming with onnxslim 0.1.70...
ONNX: export success ✅ 2.2s, saved as '/content/drive/MyDrive/Intern/Insiso/models/model_30epochs.onnx' (11.8 MB)

Export complete (2.6s)
Results saved to /content/drive/MyDrive/Intern/Insiso/models
Predict:         yolo predict task=detect model=/content/drive/MyDrive/Intern/Insiso/models/model_30epochs.onnx imgsz=640  
Validate:        yolo val task=detect model=/content/drive/MyDrive/Intern/Insiso/models/model_30epochs.onnx imgsz=640 data=/content/drive/MyDrive/Intern/Insiso/dataset/data.yaml  
Visualize:       https://netron.app
✅ Model exported to ONNX: /conte

**QUANTIZE**

In [18]:
!pip install onnx onnxruntime onnxruntime-tools

import onnx
from onnxruntime.quantization import quantize_dynamic, QuantType

onnx_model = "/content/drive/MyDrive/Intern/Insiso/models/model_30epochs.onnx"
quantized_model = "/content/drive/MyDrive/Intern/Insiso/models/model_int8.onnx"

quantize_dynamic(
    model_input=onnx_model,
    model_output=quantized_model,
    weight_type=QuantType.QInt8
)

print(f"✅ INT8 quantized model saved at: {quantized_model}")


✅ INT8 quantized model saved at: /content/drive/MyDrive/Intern/Insiso/models/model_int8.onnx


**MODEL SIZES**

In [20]:
import os

# Directory Path
model_dir = '/content/drive/MyDrive/Intern/Insiso/models'

#  Function to Calculate File Size
def get_file_size(filepath):
    size_bytes = os.path.getsize(filepath)
    size_mb = size_bytes / (1024 * 1024)
    return round(size_mb, 2)

#  Iterate Through All Model Files
print(f"{'Model Filename':<40} {'Size (MB)':>10}")
print("-" * 55)

for filename in os.listdir(model_dir):
    file_path = os.path.join(model_dir, filename)
    if os.path.isfile(file_path):
        size = get_file_size(file_path)
        print(f"{filename:<40} {size:>10} MB")


Model Filename                            Size (MB)
-------------------------------------------------------
model_30epochs.pt                              5.98 MB
model_30epochs.onnx                           11.79 MB
model_int8.onnx                                3.29 MB


**(Bonus): simple lane detection baseline using Canny + Hough**

In [ ]:
import cv2
import numpy as np
import os

# - Paths
input_folder = '/content/drive/MyDrive/Intern/Insiso/lane_images'
output_folder = '/content/drive/MyDrive/Intern/Insiso/lane_outputs'
os.makedirs(output_folder, exist_ok=True)

# ROI helper
def region_of_interest(img, vertices=None):
    if vertices is None:
        return img
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, [vertices], 255)
    return cv2.bitwise_and(img, mask)

# ---- Main loop ----
for img_file in os.listdir(input_folder):
    if not img_file.endswith('.jpg'):
        continue
    img_path = os.path.join(input_folder, img_file)
    img = cv2.imread(img_path)

 -
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    edges = cv2.Canny(blur, 50, 150)


    roi_edges = edges

    #  Hough Transform for straight lines
    lines = cv2.HoughLinesP(
        roi_edges,
        rho=1,
        theta=np.pi/180,
        threshold=50,       # minimum votes
        minLineLength=100,  # minimum length of line
        maxLineGap=20       # gap to merge line segments
    )

    # Draw lines
    line_img = np.zeros_like(img)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_img, (x1, y1), (x2, y2), (0,255,0), 3)

    # Overlay on original image
    output = cv2.addWeighted(img, 0.8, line_img, 1, 1)

    # Step 6: Save output
    cv2.imwrite(os.path.join(output_folder, img_file), output)

print("Straight lane detection complete. Check outputs in:", output_folder)


Straight lane detection complete. Check outputs in: /content/drive/MyDrive/Intern/Insiso/lane_outputs


**TEST FPS YOLO**

In [21]:
import time
from ultralytics import YOLO
import cv2

# Load YOLO model
model = YOLO('/content/drive/MyDrive/Intern/Insiso/models/model_30epochs.pt')

# Load a sample image
img = cv2.imread('/content/drive/MyDrive/Intern/Insiso/dataset/test/images/youtube-100_jpg.rf.9536b3b2e8d549b2d7961a5d782a4ded.jpg')

# Measure FPS on CPU
start = time.time()
N = 50  # number of inference runs
for _ in range(N):
    results = model(img)
end = time.time()

fps = N / (end - start)
print(f"FPS on CPU: {fps:.2f}")



0: 640x640 8 cars, 2 persons, 9.1ms
Speed: 2.8ms preprocess, 9.1ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 9.4ms
Speed: 2.3ms preprocess, 9.4ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 8.3ms
Speed: 1.9ms preprocess, 8.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 8.7ms
Speed: 1.9ms preprocess, 8.7ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 7.9ms
Speed: 2.0ms preprocess, 7.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 8.3ms
Speed: 2.1ms preprocess, 8.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 9.2ms
Speed: 1.8ms preprocess, 9.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 8.9ms
Speed: 3.0ms preprocess, 8.9ms i

**TEST FPS ONNX**

In [22]:
import time
from ultralytics import YOLO
import cv2

# Load YOLO model
model = YOLO('/content/drive/MyDrive/Intern/Insiso/models/model_30epochs.onnx')

# Load a sample image
img = cv2.imread('/content/drive/MyDrive/Intern/Insiso/dataset/test/images/youtube-100_jpg.rf.9536b3b2e8d549b2d7961a5d782a4ded.jpg')

# Measure FPS on CPU
start = time.time()
N = 50  # number of inference runs
for _ in range(N):
    results = model(img)
end = time.time()

fps = N / (end - start)
print(f"FPS on CPU: {fps:.2f}")


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading /content/drive/MyDrive/Intern/Insiso/models/model_30epochs.onnx for ONNX Runtime inference...
WARNING ⚠️ Failed to start ONNX Runtime with CUDA. Using CPU...
Using ONNX Runtime 1.23.0 CPUExecutionProvider

0: 640x640 8 cars, 2 persons, 127.7ms
Speed: 4.7ms preprocess, 127.7ms inference, 9.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 119.6ms
Speed: 3.9ms preprocess, 119.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 110.5ms
Speed: 3.1ms preprocess, 110.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 109.8ms
Speed: 2.6ms preprocess, 109.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 8 cars, 2 persons, 110.5ms
Speed: 3.2ms preprocess, 